## Capstone Project - The Battle of Neighborhoods (Week 1)

### Introduction

####  Description of the problem and a discussion of the background

Paris is the most densely populated capital city in europe and the fourth in the world
(21,498/km² - 53,000/sq mi).
As a consequence the real estate prices are amongst the highest in the world ( 8th in the world with 14,017.63 $  per square meter).

Paris being a city where relatively few people possess a car (due to excessive taxes and parking fees). So a car rental agency can be a juicy business because a lot of residents may need ponctually car

For this project let's put ourselves in the shoes of an entrepeneur looking to open a car rental agency. So for the location we have to find a balance between the real estate prices and the presence of competitors. The vicinity of a major railway station will be an asset as well




#### Data Description
To solve this problem we will use the following data :

* A csv file from "Open platform for French public data"  containing the list of all the Paris districts with their coordinates. [1]

* Foursquare API to get the car rental agencies for each district [2]

*  Another csv file from "Open platform for French public data" with the real estate prices for all the french cities from which I extracted the price for only the 20 Paris districts [3]


### References

* [1] [Paris districts coordinates](https://www.data.gouv.fr/fr/datasets/arrondissements-1/) 

* [2] [Foursquare API](https://developer.foursquare.com/)

* [3] [Average housing prices in France](https://www.data.gouv.fr/fr/datasets/prix-moyen-au-m2-des-ventes-de-maisons-et-dappartements-par-commune-en-2017/)